In [2]:
from music21 import converter,instrument,note,chord,stream
import glob
import pickle
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import *
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping,ModelCheckpoint

Using TensorFlow backend.


In [3]:
midi = converter.parse("midi_songs/EyesOnMePiano.mid")

In [4]:
print(midi)

<music21.stream.Score 0x1f95f874ac8>


In [5]:
midi.show('midi')

In [6]:
#midi.show('text')

In [7]:
elements_to_parse = midi.flat.notes

In [8]:
print(len(elements_to_parse))

1421


In [9]:
i=0
for elem in elements_to_parse:
    print(elem,elem.offset)
    i+=1
    if i==10:
        break

<music21.note.Note A> 0.0
<music21.note.Note A> 0.0
<music21.note.Note A> 0.0
<music21.note.Note A> 0.25
<music21.note.Note G> 2/3
<music21.note.Note G> 2/3
<music21.note.Note F#> 1.0
<music21.note.Note F#> 1.25
<music21.note.Note D> 1.5
<music21.note.Note D> 1.5


In [10]:
def generate_notes_and_chords(elements_to_parse,notes_demo=[]):
    for elem in elements_to_parse:
        if isinstance(elem,note.Note):
            notes_demo.append(str(elem.pitch))
        elif isinstance(elem,chord.Chord):
            notes_demo.append('+'.join([str(e) for e in elem.normalOrder]))
    return notes_demo
print(generate_notes_and_chords(elements_to_parse,[])[40:70])

['A4', 'E5', 'C#4', 'F#4', 'A4', 'E5', 'C#4', 'F#4', 'C#5', 'A3', 'C#5', 'A3', 'F#4', 'B4', 'G3', 'E4', 'F#4', 'B4', 'G3', 'E4', 'C#5', 'A3', 'C#5', 'A3', 'D5', 'B3', 'D5', 'B3', '1+6', '9+2']


In [11]:
notes = []
for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file)
    elements_to_parse = midi.flat.notes
    generate_notes_and_chords(elements_to_parse,notes)

In [12]:
print(notes[:100])

['4+9', 'E2', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', '11+4', '4+9', '11+4', '4+9', '4+9', '4+9', '4+9', '4+9', '0+4', 'E2', '4+9', '0+4', '4+9', '4+9', '4+9', '4+9', '4+9', '9+2', '4+9', '9+2', '9+2', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', 'E2', '4+9', '4+9', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '5+11', '4+9', '5+11', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '9+0', 'E2', '4+9', '9+0', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '11+2', '4+9', '11+2', '11+2', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '3+7+11', 'E-2', '3+7+11', 'B2', 'G2', '1+5+9', 'F#2', '1+5+9', '3+7+11', 'E-2', '3+7+11', 'G2', 'B2', 'E-3']


In [13]:
len(notes)

60498

In [14]:
pd.DataFrame(notes).to_csv("all_notes.csv",index=False)

In [15]:
notes = pd.read_csv("all_notes.csv").iloc[:,0].values

In [16]:
vocab = set(notes)

In [17]:
n_vocab = len(vocab)
print(n_vocab)

359


In [18]:
print("Total notes- ", len(notes))
print("Unique notes- ",  len(vocab))

Total notes-  60498
Unique notes-  359


In [19]:
sequence_length =100

In [20]:
pitch_name = sorted(vocab)

In [21]:
ele_to_int = dict((ele,num) for num,ele in enumerate(pitch_name))
int_to_ele = dict((num,ele) for num,ele in enumerate(pitch_name))

In [22]:
network_input = []
network_output = []
for i in range(len(notes)-sequence_length):
    seq_in = notes[i:i+sequence_length]
    seq_out = notes[i+sequence_length]
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append([ele_to_int[seq_out]])

In [23]:
n_patterns = len(network_input)
print(n_patterns)

60398


In [24]:
network_input = np.reshape(network_input,(n_patterns,sequence_length,1))
print(network_input.shape)
network_output = to_categorical(network_output)
print(network_output.shape)

(60398, 100, 1)
(60398, 359)


In [25]:
normalized_network_input = network_input/float(n_vocab)

In [26]:
model = Sequential()
model.add(LSTM(512,input_shape = (normalized_network_input.shape[1],normalized_network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab,activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [27]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [ ]:
checkPoint = ModelCheckpoint("best_model.hdf5",monitor='val_loss',verbose=False,save_best_only=True)
earlyStop = EarlyStopping(monitor='val_loss',verbose=False,patience=10)
history = model.fit(normalized_network_input,network_output,batch_size=64,epochs=100,callbacks=[checkPoint,earlyStop])

Epoch 1/100
 8832/60398 [===>..........................] - ETA: 47:58 - loss: 4.8695

In [28]:
model.load_weights("best_model.hdf5")

In [29]:
network_input = []
for i in range(len(notes)-sequence_length):
    seq_in = notes[i:i+sequence_length]
    network_input.append([ele_to_int[ch] for ch in seq_in])

In [30]:
start = np.random.randint(len(network_input)-1)
pattern = network_input[start]
prediction_output = []

for note_idx in range(300):
    prediction_input = np.reshape(pattern,(1,len(pattern),1))
    prediction_input = prediction_input/float(n_vocab)
    
    prediction = model.predict(prediction_input,verbose=0)
    
    idx = np.argmax(prediction)
    result = int_to_ele[idx]
    prediction_output.append(result)
    
    pattern.append(idx)
    pattern = pattern[1:]

In [31]:
print(prediction_output)

['9', '9+2', '9', '4+9', 'E3', '0+4', '9', '9', '9', '9', 'E3', 'E3', 'E3', 'B3', '9', '9+0', '9+2', '9', '11', '9', 'A2', '11', '9', 'A2', '9', '9', '9', '9', '9+0+4', '9', '9+0+4', '9+0+4', '7+11+2', '7+11+2', '7+11+2', 'A2', '9', 'B3', '9+0+4', 'A2', '9+0+4', 'A2', '7+11+2', 'A2', '7+11+2', 'A2', '7+11+2', 'A2', '6+11+0', '8', '9+0+4', 'A2', '9+0+4', 'A2', '6+11+0', '8', 'D4', 'A2', 'A3', 'G5', '10+2+5', 'G2', 'A3', 'A3', '4+9', '9', '8', '0+3', '9+0+4', '9', '0+4', '9', 'C3', 'E3', '4+9', '9', 'A2', '9', '9', '9', '9', '9+0+4', '9+0+4', '9+0+4', 'B3', '9+0+4', 'A3', '9+0+4', 'A2', 'B5', 'A2', 'A2', 'E2', 'G4', 'E2', '7+11+2', 'A2', '9+2', '9', '6+11+0', 'A2', '7+11+2', 'A2', '7+11+2', '7', '7', 'A2', '9+0+4', 'A2', '7', 'A2', '9+2', 'A2', '7', 'G3', '7', 'A2', '7', 'G3', '2+7', 'G3', '7', 'G3', '7+11', '4', '7', 'A2', '7', '7', '6+9', '4+8+11', 'C3', 'D4', '7', '7', 'D4', '9+2', '9', '7+11+2', '2', 'A2', '2+6+9', '0+4', '6+9', '2+6+9', '2', '6+9', '2+7', 'A2', '9+2', '9', '9+0+4', 

In [35]:
offset = 0
output_notes = []
for pattern in prediction_output:
    if ('+' in pattern) or (pattern.isdigit()):
        notes_in_chord = pattern.split('+')
        temp_notes = []
        for current_node in notes_in_chord:
            new_note = note.Note(int(current_node))
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
        
        new_chord = chord.Chord(temp_notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
        
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset+=float(np.random.randint(1,20))/20.0

In [36]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi',fp = 'new_song.mid')

'new_song.mid'

In [37]:
midi_stream.show('midi')